<a href="https://colab.research.google.com/github/LiChaidoescode/March-Model-Madness/blob/main/NSDC_March_Madness_Classification_Outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### March Model Madness - a straightforward approach to predictive algorithms  

In [ ]:
import pandas as pd
import math
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
import csv
import random
import numpy
import seaborn as sb

The first thing was to import the most recent regular season team-level statistics along team-level box scores for many regular seasons of historical data, starting with the 2003 season (men). 

In [ ]:
mm_tourney = pd.read_csv("mmmData/MNCAATourneyDetailedResults.csv")
mm_tourney = mm_tourney.dropna(axis = 0)
mm_tourney


In [ ]:
mm_regseason = pd.read_csv("mmmData/MRegularSeasonDetailedResults.csv")
mm_regseason = mm_regseason.dropna(axis = 0)
mm_regseason.head()

In [ ]:
corrM1 = mm_tourney.corr()
corrM2 = mm_regseason.corr()

In [ ]:
sb.heatmap(corrM1, cmap = "Blues")

In [ ]:
sb.heatmap(corrM2, cmap = "crest")

This correlation matrix only represents numerical values in the data, so things such as team names and rounds in the bracket are not displayed. I followed this step from the NSDC March Madness Classification outline. By inspection, most of the correlated box level scores are quite obvious. For example, field goals made by a losing team is pretty highly correlated with defensive rebounds made by the winning team. Originally, my hopes were to determine which statistics would be the most highly correlated to create a sleeker model. 

After reviewing a number of old march madness projects I decided to take the path taken by Matt Harvey. He used recent team performance and ELO ratings to predict tournament performance. I've used his work as a starting point, updated the data used, and incorporated additional statistical learning models. According to my research, support vector machines are better for this type of highly variable data. 

In [ ]:
# Initalize a few of the variables and arrays that are used by multiple functions. 
base_elo = 1600
team_elos = {}  # Reset each year.
team_stats = {}
X = []
y = []
submission_data = []
seeds = pd.read_csv('mmmData/MNCAATourneySeeds.csv')
names = pd.read_csv('mmmData/MTeams.csv')
seeds = seeds.merge(names[['TeamName']], on='TeamID')
prediction_year = 2023
folder = 'mmmData'

In [ ]:
def initialize_data():
    for i in range(1985, prediction_year+1):
        team_elos[i] = {}
        team_stats[i] = {}

In [ ]:
# The purpose of this function is to retrieve the ELO ranking of a specific team. 
# If an error occurs on the return, the function retrives the previous season's ending
# value. If an error occurs there it assisngs a base elo ranking. 
def get_elo(season, team):
    try:
        return team_elos[season][team]
    except:
        try:
            # Get the previous season's ending value.
            team_elos[season][team] = team_elos[season - 1][team]
            return team_elos[season][team]
        except:
            # Get the starter elo.

            team_elos[season][team] = base_elo
            
            return team_elos[season][team]

In [ ]:
# Given two teams and the season, this function determines the ELO rating. 
# The link is broken but I got this function from Matt Harvey's work. 
def calc_elo(win_team, lose_team, season):
    winner_rank = get_elo(season, win_team)
    loser_rank = get_elo(season, lose_team)

    """
    This is originally from from:
    http://zurb.com/forrst/posts/An_Elo_Rating_function_in_Python_written_for_foo-hQl
    """
    
    rank_diff = winner_rank - loser_rank
    exp = (rank_diff * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    if winner_rank < 2100:
        k = 32
    elif winner_rank >= 2100 and winner_rank < 2400:
        k = 24
    else:
        k = 16
    new_winner_rank = round(winner_rank + (k * (1 - odds)))
    new_rank_diff = new_winner_rank - winner_rank
    new_loser_rank = loser_rank - new_rank_diff

    return new_winner_rank, new_loser_rank


In [ ]:
def get_stat(season, team, field):
    try:
        l = team_stats[season][team][field]
        return sum(l) / float(len(l))
    except:
        return 0

In [ ]:
# This function is used to predict the winner of two teams in a head to head match up. 
# An array called features is created locally, the next step is to collect the elo rating and 
# stats of that team. 
def predict_winner(team_1, team_2, model, season, stat_fields):
    features = []

    # Team 1
    features.append(get_elo(season, team_1))
    for stat in stat_fields:
        features.append(get_stat(season, team_1, stat))

    # Team 2
    features.append(get_elo(season, team_2))
    for stat in stat_fields:
        features.append(get_stat(season, team_2, stat))

    return model.predict_proba([features])

In [ ]:
def update_stats(season, team, fields):
    """
    This accepts some stats for a team and udpates the averages.
    First, we check if the team is in the dict yet. If it's not, we add it.
    Then, we try to check if the key has more than 5 values in it.
        If it does, we remove the first one
        Either way, we append the new one.
    If we can't check, then it doesn't exist, so we just add this.
    Later, we'll get the average of these items.
    """
    if team not in team_stats[season]:
        team_stats[season][team] = {}

    for key, value in fields.items():
        # Make sure we have the field.
        if key not in team_stats[season][team]:
            team_stats[season][team][key] = []

        if len(team_stats[season][team][key]) >= 9:
            team_stats[season][team][key].pop()
        team_stats[season][team][key].append(value)

In [ ]:
def build_team_dict():
    team_ids = seeds
    team_id_map = {}
    for index, row in team_ids.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    return team_id_map

In [ ]:
def build_season_data(all_data):
    
    # Calculate the elo rating for every game for every team, each season.
    # Store the elo per season so we can retrieve their end elo
    # later in order to predict the tournaments without having to
    # inject the prediction into this loop.
    print("Building season data.")
    for index, row in all_data.iterrows():
        # Used to skip matchups where we don't have usable stats yet.
        skip = 0

        # Get starter or previous elos.
        team_1_elo = get_elo(row['Season'], row['WTeamID'])
        team_2_elo = get_elo(row['Season'], row['LTeamID'])

        # Add 100 to the home team (# taken from Nate Silver analysis.) 
        # Nate Silver is one of the founders of FiveThirtyEight and initiated a
        # data driven approach to predicting the team with the highest probability 
        # of winning March Madness. This line of code quantifies "home court advantage"

        if row['WLoc'] == 'H':
            team_1_elo += 100
        elif row['WLoc'] == 'A':
            team_2_elo += 100

        # We'll create some arrays to use later.
        team_1_features = [team_1_elo]
        team_2_features = [team_2_elo]

        # Build arrays out of the stats we're tracking.
        for field in stat_fields:
            team_1_stat = get_stat(row['Season'], row['WTeamID'], field)
            team_2_stat = get_stat(row['Season'], row['LTeamID'], field)
            if team_1_stat != 0 and team_2_stat != 0:
                team_1_features.append(team_1_stat)
                team_2_features.append(team_2_stat)
            else:
                skip = 1

        if skip == 0:  # Make sure we have stats.
            # Randomly select left and right and 0 or 1 so we can train
            # for multiple classes.
            if random.random() > 0.5:
                X.append(team_1_features + team_2_features)
                y.append(0)
            else:
                X.append(team_2_features + team_1_features)
                y.append(1)

        # AFTER we add the current stuff to the prediction, update for
        # next time. Order here is key so we don't fit on data from the
        # same game we're trying to predict.
        if row['WFTA'] != 0 and row['LFTA'] != 0:
            stat_1_fields = {
                'score': row['WScore'],
                'fgp': row['WFGM'] / row['WFGA'] * 100,
                'fga': row['WFGA'],
                'fga3': row['WFGA3'],
                '3pp': row['WFGM3'] / row['WFGA3'] * 100,
                'ftp': row['WFTM'] / row['WFTA'] * 100,
                'or': row['WOR'],
                'dr': row['WDR'],
                'ast': row['WAst'],
                'to': row['WTO'],
                'stl': row['WStl'],
                'blk': row['WBlk'],
                'pf': row['WPF']
            }
            stat_2_fields = {
                'score': row['LScore'],
                'fgp': row['LFGM'] / row['LFGA'] * 100,
                'fga': row['LFGA'],
                'fga3': row['LFGA3'],
                '3pp': row['LFGM3'] / row['LFGA3'] * 100,
                'ftp': row['LFTM'] / row['LFTA'] * 100,
                'or': row['LOR'],
                'dr': row['LDR'],
                'ast': row['LAst'],
                'to': row['LTO'],
                'stl': row['LStl'],
                'blk': row['LBlk'],
                'pf': row['LPF']
            }
            update_stats(row['Season'], row['WTeamID'], stat_1_fields)
            update_stats(row['Season'], row['LTeamID'], stat_2_fields)

        # Now that we've added them, calc the new elo.
        new_winner_rank, new_loser_rank = calc_elo(
            row['WTeamID'], row['LTeamID'], row['Season'])
        team_elos[row['Season']][row['WTeamID']] = new_winner_rank
        team_elos[row['Season']][row['LTeamID']] = new_loser_rank

    
    
    
    return X, y

In [ ]:
if __name__ == "__main__":
    stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']

    initialize_data()
    season_data = mm_regseason
    tourney_data = mm_tourney
    frames = [season_data, tourney_data]
    all_data = pd.concat(frames)

    # Build the working data.
    X, y = build_season_data(all_data)

    # Fit the model with our training data. 
    # The newton-cholesky solver was used because according to sci-kit learn documentation
    # this solver is ideal for classifcation problems where the number of samples (1248) is
    # much larger than the number of features (34)

    print("Fitting on %d samples." % len(X))
    model = LogisticRegression(solver='newton-cholesky')
    model.fit(X, y)
    kfold = KFold(n_splits=5, random_state=0, shuffle=True)
    results = cross_val_score(model, X, y, cv=kfold)
    
    # Output the accuracy. Calculate the mean and std across all folds.
    print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

    # Now predict tournament matchups.
    print("Getting teams.")
    # for i in range(2016, 2017):
    tourney_teams = []
    for index, row in seeds.iterrows():
        if row['Season'] == prediction_year:
            tourney_teams.append(row['TeamID'])

    # Build our prediction of every matchup.
    print("Predicting matchups.")
    tourney_teams.sort()
    for team_1 in tourney_teams:
        for team_2 in tourney_teams:
            if team_1 < team_2:
                prediction = predict_winner(
                    team_1, team_2, model, prediction_year, stat_fields)
                label = str(prediction_year) + '_' + str(team_1) + '_' + \
                    str(team_2)
                submission_data.append([label, prediction[0][0]])


                # Write the results.
    print("Writing %d results." % len(submission_data))
    with open(folder + '/submission.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'pred'])
        writer.writerows(submission_data)

    # Now so that we can use this to fill out a bracket, create a readable
    # version.
    print("Outputting readable results.")
    team_id_map = build_team_dict()
    readable = []
    less_readable = []  # A version that's easy to look up.
    for pred in submission_data:
        parts = pred[0].split('_')
        less_readable.append(
            [team_id_map[float(parts[1])], team_id_map[float(parts[2])], pred[1]])
        # Order them properly.
        if pred[1] > 0.5:
            winning = float(parts[1])
            losing = float(parts[2])
            proba = pred[1]
        else:
            winning = float(parts[2])
            losing = float(parts[1])
            proba = 1 - pred[1]
        readable.append(
            [
                '%s beats %s: %f' %
                (team_id_map[winning], team_id_map[losing], proba)
            ]
        )
    with open(folder + '/readable-predictions.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(readable)
    with open(folder + '/less-readable-predictions.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(less_readable)

Then we want to select any variables that we do not think will be helpful. In this case, I am choosing to remove the year variables, as I only want to focus on a team's game stats, and not the year that they took place, in order to predict how far teams will go long term in the game. You can select what variables you want to include in your model by thinking through your assumptions about what variables are most highly correlated. You can even use a correlation matrix to best determine the highest correlations between each variable and your target variable.

In [ ]:
#mm = dummies.drop('YEAR', axis = 1)

Now we want to consider what method of classification we are going to use. There are many different types classification methods. The following are just a subset of them:
- Decision Tree/Random Forest: dataset attributes become nodes or branches of a tree 
    - Pros: works well with both numerical and categorical data, implicitly performs feature selection, not greatly influenced by outliers
    - Cons: not easily interpretable, computationally intensive
- K-Nearest Neighbors: creates groups based off of the k-nearest values to some centroid
    - Pros: very simple, easy to implement for multi-class problems, many distance variables to choose from
    - Cons: very sensitive to outliers, slow and performs worse for higher dimensions of data
- Support Vector Machines: model with associated learning algorithms that analyze data for classifications
    - Pros: works well with clear margin of separation, effective in high dimensions
    - Cons: doesn't perform well on noisy data, no probability estimates

For this demonstration, we are going to work with a random forest algorithm. We need to start by determining what the best parameters are based on our data for our random forest algorithm. To do this we are going to create a pipeline that contains the algorithm and parameters we want to train, followed by a grid search which will test a variety of values for each and return the best fit on the training data.

In [ ]:
#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.decomposition import PCA

In [ ]:
#pipe = Pipeline([['reduction', PCA()], ['classify', RandomForestClassifier()]])
#pipe_grid = { 'reduction__n_components': [2, 5, 8, 10], 'classify__n_estimators': [10, 100, 1000], 'classify__criterion': ["gini", "entropy", "log_loss"]}
#gs = GridSearchCV(pipe, pipe_grid)
#gs.fit(X_train, y_train)
#best_parameters = gs.best_params_

In [ ]:
#best_parameters

Once we have the best parameters, we want to be able to test our model using our X_test and y_test values. We start by reducing the dimensions of our data using the best computed parameter. We then fit the X_train and y_train data to the RandomForestClassifier with the values that we received from the GridSearchCV best parameters. We then predict on the X_test data to get a prediction for y.

In [ ]:
#pca = PCA(n_components = best_parameters['reduction__n_components'])
#X_train = pca.fit_transform(X_train)

In [ ]:
#model = RandomForestClassifier(n_estimators = best_parameters['classify__n_estimators'], criterion = best_parameters['classify__criterion'])
#fitted_model = model.fit(X_train, y_train)
#y_predicted = fitted_model.predict(pca.fit_transform(X_test))

Now we want to test how well our algorithm performed on our testing data. We can do this in many different ways. One way is by computing the accuracy and precision scores using the sklearn.metrics library. The definition of both are as follows:
- Accuracy Score: an evaluation metric that measures the number of correct predictions made by a model in relation to the total number of predictions made
- Precision Score: the ratio of true positive to the sum of true positive and false positive

Another great way to determine the intricacies of how our model is doing is by generating the confusion matrix which shows both how many times the model got the right answer for each classification, but also how many misclassifications occured per category.

In [ ]:
#from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
#cm = confusion_matrix(y_test, y_predicted, labels = mm['POSTSEASON'].unique())
#acc = accuracy_score(y_test, y_predicted)
#prec = precision_score(y_test, y_predicted, average = "weighted")
#print("Accuracy Score: " + str(acc))
#print("Precision Score: " + str(prec))

In [ ]:
#sb.heatmap(cm, annot = True, cmap="Blues", xticklabels = mm['POSTSEASON'].unique(), yticklabels = mm['POSTSEASON'].unique())